# Data Analysis (Required)

## Step 1: Import Libraries and Prepare Dataset for analysis.

The below cell: 

1. Imports pandas and plotly libraries for analysis and visualization
2. Imports our CSV into a dataframe with the assumption that this noteboook is being run in a directory with a subdirector named 'files' which contains our csv.

In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('files/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv')


/var/folders/3k/phn3qb093cj5b3twlzg9smrh0000gq/T/ipykernel_18053/2446485954.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('files/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv')


## Step 2: Data Exploration

The below cells will: 

1. Tell us the names of the columns in our data.
2. The datatypes in said columns.
3. Check for missing values. 
4. Check for duplicate rows.
5. Provide some statistical summary in our data.
6. Count the number of occurunces of unique values in our data's columns. 

In [2]:
df.columns

Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes'],
      dtype='object')

In [3]:
# Check data types
df.dtypes


Prscrbr_NPI                int64
Prscrbr_Last_Org_Name     object
Prscrbr_First_Name        object
Prscrbr_City              object
Prscrbr_State_Abrvtn      object
Prscrbr_State_FIPS        object
Prscrbr_Type              object
Prscrbr_Type_Src          object
Brnd_Name                 object
Gnrc_Name                 object
Tot_Clms                   int64
Tot_30day_Fills          float64
Tot_Day_Suply              int64
Tot_Drug_Cst             float64
Tot_Benes                float64
GE65_Sprsn_Flag           object
GE65_Tot_Clms            float64
GE65_Tot_30day_Fills     float64
GE65_Tot_Drug_Cst        float64
GE65_Tot_Day_Suply       float64
GE65_Bene_Sprsn_Flag      object
GE65_Tot_Benes           float64
dtype: object

In [4]:
# Check for missing values
df.isnull().sum()


Prscrbr_NPI                     0
Prscrbr_Last_Org_Name           0
Prscrbr_First_Name              1
Prscrbr_City                    2
Prscrbr_State_Abrvtn            0
Prscrbr_State_FIPS              0
Prscrbr_Type                   56
Prscrbr_Type_Src                0
Brnd_Name                       0
Gnrc_Name                       0
Tot_Clms                        0
Tot_30day_Fills                 0
Tot_Day_Suply                   0
Tot_Drug_Cst                    0
Tot_Benes                14503885
GE65_Sprsn_Flag          13996182
GE65_Tot_Clms            11235680
GE65_Tot_30day_Fills     11235680
GE65_Tot_Drug_Cst        11235680
GE65_Tot_Day_Suply       11235680
GE65_Bene_Sprsn_Flag      2655545
GE65_Tot_Benes           22576317
dtype: int64

In [5]:
# Check for duplicate rows
df.duplicated().sum()


0

In [6]:
# Statistical summary
df.describe()


,Prscrbr_NPI,Tot_Clms,Tot_30day_Fills,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Tot_Benes
count,2.523186e+07,2.523186e+07,2.523186e+07,2.523186e+07,2.523186e+07,1.072798e+07,1.399618e+07,1.399618e+07,1.399618e+07,1.399618e+07,2.655545e+06
mean,1.499429e+09,5.117672e+01,9.305323e+01,2.638394e+03,6.508675e+03,3.049494e+01,4.978447e+01,9.373649e+01,6.144047e+03,2.685797e+03,3.311168e+01
std,2.877561e+08,1.124580e+02,1.866139e+02,5.125924e+03,4.631451e+04,8.909593e+01,1.259832e+02,2.053814e+02,4.516862e+04,5.499884e+03,1.597984e+02
min,1.003000e+09,1.100000e+01,1.100000e+01,1.100000e+01,0.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.245726e+09,1.500000e+01,2.200000e+01,5.100000e+02,2.555600e+02,1.400000e+01,1.300000e+01,1.900000e+01,2.199500e+02,4.570000e+02,1.300000e+01
50%,1.497946e+09,2.400000e+01,4.000000e+01,1.110000e+03,6.557500e+02,2.000000e+01,2.200000e+01,3.800000e+01,6.057900e+02,1.080000e+03,1.900000e+01
75%,1.740736e+09,5.100000e+01,8.800000e+01,2.520000e+03,2.098330e+03,3.400000e+01,4.700000e+01,8.500000e+01,1.986370e+03,2.430000e+03,3.400000e+01
max,1.993000e+09,1.737860e+05,1.739408e+05,9.395990e+05,3.231571e+07,1.342230e+05,1.630540e+05,1.631887e+05,3.030928e+07,9.311090e+05,1.257890e+05


In [7]:
# Check unique values in the 'Prscrbr_State_Abrvtn' column. The name of the column in between the [''] can be replaced
#as we move through our analysis. 
df['Prscrbr_State_Abrvtn'].value_counts()


CA    2327261
FL    1787374
TX    1650984
NY    1650963
PA    1232487
       ...   
AE        592
AP        512
MP        251
AA        106
AS          6
Name: Prscrbr_State_Abrvtn, Length: 61, dtype: int64

## Data Analysis

The two cells below  will create a new dataframe from our full dataset by checking for any rows that contain the value 'buprenorphine' in either the Brand Name column (Brnd_Name in our data) or the Generic Name column (Gnrc_Name in our data).

The reason the numbers in the second cell are greater than in the first (though they aggregate based on the same criteria) is that the first column will find the rows in which the generic/brand name columsn contain buprenorphine, and drop duplicates. The second cell doesn't do this.

I couldn't find whether or not this is the correct way to go about it in the methodology document so I wanted to provide both counts. 

In [6]:
# Identify rows that describe claims for buprenorphine
# If buprenorphine is a brand name
buprenorphine_brand_df_d = df[df['Brnd_Name'].str.contains('buprenorphine', na=False, case=False)]
# If buprenorphine is a generic name
buprenorphine_generic_df_d = df[df['Gnrc_Name'].str.contains('buprenorphine', na=False, case=False)]

# Concatenate the two dataframes and remove duplicates
buprenorphine_df_d = pd.concat([buprenorphine_brand_df, buprenorphine_generic_df], ignore_index=True).drop_duplicates()

# The 'Tot_Day_Suply' field can be used to describe per state usage of buprenorphine, as it represents the total number of days' supply prescribed for the drug.
# Aggregate per state usage of buprenorphine
state_usage_d = buprenorphine_df.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum()

# Top 5 states by volume of buprenorphine usage
top_states_d = state_usage.sort_values(ascending=False).head(5)
print(top_states_d)

Prscrbr_State_Abrvtn
CA    3244694
PA    2880656
FL    2671723
MA    2447394
NC    2382440
Name: Tot_Day_Suply, dtype: int64


In [7]:
# Identify rows that describe claims for buprenorphine
# If buprenorphine is a brand name
buprenorphine_brand_df = df[df['Brnd_Name'].str.contains('buprenorphine', na=False, case=False)]
# If buprenorphine is a generic name
buprenorphine_generic_df = df[df['Gnrc_Name'].str.contains('buprenorphine', na=False, case=False)]

# Concatenate the two dataframes and remove duplicates
buprenorphine_df = pd.concat([buprenorphine_brand_df, buprenorphine_generic_df], ignore_index=True)

# The 'Tot_Day_Suply' field can be used to describe per state usage of buprenorphine, as it represents the total number of days' supply prescribed for the drug.
# Aggregate per state usage of buprenorphine
state_usage = buprenorphine_df.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum()

# Top 5 states by volume of buprenorphine usage
top_states = state_usage.sort_values(ascending=False).head(5)
print(top_states)


Prscrbr_State_Abrvtn
CA    5686826
PA    5197242
FL    4824105
NY    4232077
MA    4198419
Name: Tot_Day_Suply, dtype: int64


# Data Visualization (optional)

## The below two cells will take the same data and produce a produce a graphic visualization for a general audience that illustrates buprenorphine usage for all 50 states, accentuating the top five states by usage.

In [15]:
# Sum total days supply by state
state_usage_d = buprenorphine_df_d.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum().reset_index()
state_usage_d.columns = ['State', 'Total Usage']

# Sort by Total Usage
state_usage_d = state_usage_d.sort_values('Total Usage', ascending=False)

# Create a new column to determine color based on whether the state is in the top 5
state_usage_d['color'] = ['Top 5' if i < 5 else 'Other' for i in range(state_usage_d.shape[0])]

# Create the bar chart
fig = px.bar(state_usage_d, x='State', y='Total Usage', color='color',
             title='Buprenorphine Usage by State: Duplicates Dropped', labels={'Total Usage': 'Total Usage (days supply)', 'State': ''},
             color_discrete_map={'Top 5': 'red', 'Other': 'blue'})

# Show the figure
fig.show()

In [17]:
# Sum total days supply by state
state_usage = buprenorphine_df.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum().reset_index()
state_usage.columns = ['State', 'Total Usage']

# Sort by Total Usage
state_usage = state_usage.sort_values('Total Usage', ascending=False)

# Create a new column to determine color based on whether the state is in the top 5
state_usage['color'] = ['Top 5' if i < 5 else 'Other' for i in range(state_usage.shape[0])]

# Create the bar chart
fig = px.bar(state_usage, x='State', y='Total Usage', color='color',
             title='Buprenorphine Usage by State: Duplicates not Dropped', labels={'Total Usage': 'Total Usage (days supply)', 'State': ''},
             color_discrete_map={'Top 5': 'red', 'Other': 'blue'})

# Show the figure
fig.show()


In [18]:
import plotly.io as pio

# ... Create your plotly figure here ...

fig.write_image('figure_.png')


In [14]:
fig.write_html("figure.html")


### Notes and Assumptions: 

1. As mentioned 